### Importo las librerias a usar

In [1]:
import pandas as pd
import unicodedata
%matplotlib inline

### Dataset
Link a dataset: https://www.kaggle.com/zynicide/wine-reviews

In [2]:
data = pd.read_csv('wine.csv')

### Creo un dataset con todos los datos de Argentina y dejo solo las columnas que voy  usar

In [3]:
data = data[data.country == 'Argentina']
columnas = data.columns.tolist()
columnas_innecesarias = ["Unnamed: 0", "country", "description", "designation", "taster_name", "taster_twitter_handle", "region_2"]
subconjunto_lista_filtrada = [x for x in columnas if x not in columnas_innecesarias]
data = data[subconjunto_lista_filtrada]

### Verifico si hay valores nulos

In [4]:
data.isnull().sum()

points       0
price       44
province     0
region_1    53
title        0
variety      0
winery       0
dtype: int64

In [5]:
antes_de_drop = data.shape[0]

### Elimino las filas duplicadas

In [6]:
data = data.drop_duplicates()

despues_de_drop = data.shape[0]
data = data.reset_index()

### Completo las regiones NaN con ''

In [7]:
data.region_1 = data.region_1.fillna(value= '')

### Creo el campo categoria donde hay 3 tipos segun el puntaje del vino <br> <span style="color: red">0 <= 60 --- Detractor</span> <br> <span style="color: orange">61 <= 80 --- Neutro</span> <br> <span style="color: green">81 <= 100 --- Promotor</span>

In [8]:
for i in range(0, len(data)):
    if 0 <= data.loc[i, 'points'] <= 60:
        data.loc[i,'categoria'] = 'Detractor'
    elif 61 <= data.loc[i, 'points'] <= 80:
        data.loc[i,'categoria'] = 'Neutro'
    elif 81 <= data.loc[i, 'points'] <= 100:
        data.loc[i,'categoria'] = 'Promotor'
    else:
        data.loc[i,'categoria'] = ''

### Verifico cada una de las columnas, si el tipo es 0 (string) le limpio los carecteres especiales, le saco las tildes y los paso a minuscula

In [9]:
for x in range (0, len(data.columns)):
    if data.dtypes[x] == 'O':
        data[data.columns[x]] = data[data.columns[x]].str.lower().str.strip()
        data[data.columns[x]] = data[data.columns[x]].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') 

### Muestro cuantos datos habia y cuantos quedaron luego de eliminar las filas duplicadas

In [10]:
antes_de_drop, despues_de_drop

(3800, 3544)

### Verifico las regiones y provincias

In [11]:
data.region_1.unique()

array(['cafayate', 'mendoza', 'salta', 'lujan de cuyo', 'san juan', '',
       'uco valley', 'perdriel', 'tupungato', 'valle de uco',
       'calchaqui valley', 'vista flores', 'maipu', 'patagonia',
       'alto valle del rio negro', 'la consulta', 'agrelo', 'neuquen',
       'rio negro valley', 'la rioja', 'famatina valley', 'argentina',
       'tulum valley', 'buenos aires', 'jujuy', 'san carlos',
       'pedernal valley', 'zonda valley', 'medrano', 'altos de mendoza',
       'san rafael', 'catamarca'], dtype=object)

In [12]:
data.province.unique()

array(['other', 'mendoza province'], dtype=object)

### Cambio la provincia mendoza province a mendoza

In [13]:
data.province = data.province.str.replace('mendoza province', 'mendoza')

### Cambio las regiones que estan en ingles a castellano

In [14]:
data.replace({'region_1' : { 'uco valley' : 'valle de uco', 'calchaqui valley' : 'valle calchaqui', 
                            'rio negro valley' : 'alto valle del rio negro', 
                            'famatina valley'  : 'valle de famatina', 'tulum valley'  : 'oasis del tulum',
                            'pedernal valley'  : 'valle de pedernal', 'zonda valley'  : 'oasis de zonda' }}, inplace = True)

### Me fijo que las regiones hayan quedado ok

In [15]:
data.region_1.unique()

array(['cafayate', 'mendoza', 'salta', 'lujan de cuyo', 'san juan', '',
       'valle de uco', 'perdriel', 'tupungato', 'valle calchaqui',
       'vista flores', 'maipu', 'patagonia', 'alto valle del rio negro',
       'la consulta', 'agrelo', 'neuquen', 'la rioja',
       'valle de famatina', 'argentina', 'oasis del tulum',
       'buenos aires', 'jujuy', 'san carlos', 'valle de pedernal',
       'oasis de zonda', 'medrano', 'altos de mendoza', 'san rafael',
       'catamarca'], dtype=object)

### Una de las regiones es Argentina <br> Usando la vineria puedo obtener de que provincias son por lo que armo dos listas con las provincias y vinerias unicas donde la region sea argentina <br> Luego con un for voy recorriendo y donde la region era Argentina lo reemplazo por la provincia donde esta la vineria

In [16]:
lista_provincia = ['cordoba','mendoza','mendoza','mendoza','neuquen','mendoza','mendoza','mendoza','mendoza','mendoza','catamarca','san juan','mendoza','mendoza','mendoza','mendoza','mendoza']
lista_vineria = ['bodega las canitas','haarth','terrazas de los andes','san huberto','familia schroeder','rutini','mendoza vineyards','aguijon de abeja','cave extreme','ruca malen','alto 3','finca las moras','casa de campo','battle axe','lost vineyards','staphyle','trapiche']

In [17]:
for x in range (0, len(lista_provincia)):
    for i in range(0, len(data)):
        if data.winery[i] == lista_vineria[x]:
            data.loc[i,'region_1'] = lista_provincia[x]

### Hay provincia llamada other por lo que al tener ya las regiones completas reemplazo ese other por la region que le corresponde

In [18]:
for x in range (0, len(data)):
    if data.province[x] == 'other':
        data.loc[x,'province'] = data.region_1[x]

### Hay algunas regiones vacias por lo que las reemplazo por la provincia que le corresponde

In [19]:
for x in range(0, len(data)):
    if data.region_1[x] == '':
        data.loc[x,'region_1'] = data.loc[x,'province']

### Hay provincias que estan con el dato de la region como por ejemplo Cafayate cuando tiene que ser Salta. <br> Ya que tengo las regiones completas elimino la columna provincia

In [20]:
data.drop('province', axis=1, inplace=True)

### Creo un mapeo donde detallo a que provincia corresponden las regiones que salen en las provincias

In [21]:
mapeo = {
    'cafayate' : 'salta',
    'lujan de cuyo'	: 'mendoza',
    'valle de uco' : 'mendoza',
    'perdriel' : 'mendoza',
    'tupungato'	: 'mendoza',
    'valle calchaqui' : 'valle calchaqui',
    'vista flores' : 'mendoza',
    'maipu'	: 'maipu',
    'patagonia'	: 'patagonia',
    'alto valle del rio negro' : 'rio negro',
    'la consulta' : 'mendoza',
    'agrelo' : 'mendoza',
    'valle de famatina'	: 'la rioja',
    'oasis del tulum' : 'san juan',
    'san carlos' : 'salta',
    'oasis de zonda' : 'san juan',
    'medrano' : 'mendoza',
    'valle de pedernal'	: 'san juan',
    'altos de mendoza' : 'mendoza',
    'san rafael' : 'mendoza',
    'mendoza' : 'mendoza',
    'salta' : 'salta',
    'san juan' : 'san juan',
    'neuquen' : 'neuquen',
    'la rioja' : 'la rioja',
    'cordoba' : 'cordoba',
    'jujuy' : 'jujuy',
    'catamarca' : 'catamarca'
}

### Y con ese diccionario hago un map creando la columna provincia

In [22]:
data['province'] = data['region_1'].map(mapeo)

### Ahora ya tengo las provincias y las regiones bien <br> Ej: Si es region Cafayate es provincia Salta

In [23]:
data.head(10)

,index,points,price,region_1,title,variety,winery,categoria,province
0,16,87,30.0,cafayate,felix lavaque 2010 felix malbec (cafayate),malbec,felix lavaque,promotor,salta
1,17,87,13.0,mendoza,gaucho andino 2011 winemaker selection malbec ...,malbec,gaucho andino,promotor,mendoza
2,183,88,12.0,salta,alamos 2007 torrontes (salta),torrontes,alamos,promotor,salta
3,224,90,22.0,lujan de cuyo,mendel 2014 lunta malbec (lujan de cuyo),malbec,mendel,promotor,mendoza
4,231,85,10.0,mendoza,andean sky 2007 bonarda (mendoza),bonarda,andean sky,promotor,mendoza
5,245,85,12.0,san juan,finca las moras 2007 reserve chardonnay (san j...,chardonnay,finca las moras,promotor,san juan
6,253,85,15.0,lujan de cuyo,cueva de las manos 2007 reserve malbec (lujan ...,malbec,cueva de las manos,promotor,mendoza
7,261,89,37.0,mendoza,algodon 2008 estate blend gran reserva red,red blend,algodon,promotor,mendoza
8,266,89,14.0,mendoza,kaiken 2008 corte malbec-bonarda-petit verdot ...,red blend,kaiken,promotor,mendoza
9,273,89,19.0,valle de uco,o. fournier 2007 b crux red (uco valley),red blend,o. fournier,promotor,mendoza


### Verifico las provincias unicas para ver si esta todo ok <br> Encuentro 3 provincias que hay que arreglar ya que no son provincias: Valle Calchaqui, Maipu, Patagonia

In [24]:
data.province.unique()

array(['salta', 'mendoza', 'san juan', 'valle calchaqui', 'maipu',
       'patagonia', 'rio negro', 'neuquen', 'la rioja', 'cordoba',
       'jujuy', 'catamarca'], dtype=object)

### Busco todas las vinerias de esas 3 provincias <br> Googleo de donde son (unas 42)

In [25]:
vineria_provincia = ['aniello','humberto canale','bodega chacra','bodega noemia de patagonia','bodega nqn','bodega patritti','denario','ruta 22','alpataco','bodega del fin del mundo','fabre montmayou','nemesio','restivo','schroeder estate','bodegas fabre','manos negras','southern edge','valle azul','nieto senetiner','infinitus','verum','aymara','bodega don bosco','pascual toso','luigi bosca','tercos','viamonte','coiron','palo alto','finca los olmos','zuccardi','el esteco','michel torino','colome','amalaya','inca','bodega tacuil']
provincia_vineria = ['rio negro','rio negro','rio negro','rio negro','neuquen','neuquen','rio negro','mendoza','neuquen','neuquen','mendoza','neuquen','mendoza','neuquen','mendoza','mendoza','neuquen','mendoza','mendoza','rio negro','rio negro','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','salta','salta','salta','salta','salta','salta']

### Ahora recorriendo si la vineria es igual a la vineria_provincia lo que hago es reemplazar la provincia por la posicion que corresponda en provincia_vineria <br> Ej la posicion 0 es vineria aniello como es igual a la primer posicion de vineria_provincia a data.provincia en la posicion 0 la relleno con la provincia_vineria en al posicion 0 o sea: Rio Negro

In [26]:
for x in range (0, len(vineria_provincia)):
    for i in range(0, len(data)):
        if data.winery[i] == vineria_provincia[x]:
            data.loc[i,'province'] = provincia_vineria[x]

### Ahora al verificar las provincias vemos que esta todo ok

In [27]:
data.province.unique()

array(['salta', 'mendoza', 'san juan', 'rio negro', 'neuquen', 'la rioja',
       'cordoba', 'jujuy', 'catamarca'], dtype=object)

### Verifico los nulos

In [28]:
data.isnull().sum()

index         0
points        0
price        43
region_1      0
title         0
variety       0
winery        0
categoria     0
province      0
dtype: int64

### Completo todos los precios nulos con 0, al ser float los rellena con 0.0

In [29]:
data.price = data.price.fillna(value=0)

### Para obtener un precio a rellenar lo que hago es ir recorriendo todas las filas donde el precio sea igual a 0.0 y creo una variable donde van a estar todos los precios de los vinos segun variedad, provincia, region_1 y vineria y de eso obtengo la media y relleno el precio segun corresponda <br> Ej: Si la variedad es Malbec, la provincia es Mendoza, la region es Cafayate y la vineria es Felix Lavaque, la media del precio de todos los vinos que cumplan con esas condiciones va a ser el dato con el que voy a rellenar los price 0.0 donde todos los otros campos cumplan las mismas condiciones

In [30]:
data.replace({'region_1' : { 'uco valley' : 'valle de uco', 'calchaqui valley' : 'valle calchaqui', 
                            'rio negro valley' : 'alto valle del rio negro', 
                            'famatina valley'  : 'valle de famatina', 'tulum valley'  : 'oasis del tulum',
                            'pedernal valley'  : 'valle de pedernal', 'zonda valley'  : 'oasis de zonda' }}, inplace = True)

In [31]:
data.head(1)

,index,points,price,region_1,title,variety,winery,categoria,province
0,16,87,30.0,cafayate,felix lavaque 2010 felix malbec (cafayate),malbec,felix lavaque,promotor,salta


### Filtro un vino en base a multiples caracteristicas para poder ver el que tiene precio 0.0 y poder calcular manualmente la media <br> En este ejemplo el 0.0 se tendria que cambiar por un 15 ya que la media da 15.33 y redondea. (Ej mismo filtro se ve mas abajo dsp e aplicar la funcion)

In [32]:
datos = data[(data.variety == 'chardonnay') & 
    (data.province == 'mendoza') & 
    (data.region_1 == 'valle de uco') &
    (data.winery == 'andeluna')]
datos

,index,points,price,region_1,title,variety,winery,categoria,province
513,17155,84,0.0,valle de uco,andeluna 2013 1300 chardonnay (uco valley),chardonnay,andeluna,promotor,mendoza
2151,77855,85,15.0,valle de uco,andeluna 2011 1300 chardonnay (uco valley),chardonnay,andeluna,promotor,mendoza
2476,90905,80,16.0,valle de uco,andeluna 2015 1300 chardonnay (valle de uco),chardonnay,andeluna,neutro,mendoza
2550,93677,84,15.0,valle de uco,andeluna 2012 1300 chardonnay (uco valley),chardonnay,andeluna,promotor,mendoza


In [33]:
for x in range(0, len(data)):
    if data.price[x] == 0.0:
        filtro = data[(data.variety == data.variety[x]) & 
                      (data.province == data.province[x]) & 
                      (data.region_1 == data.region_1[x]) &
                      (data.winery == data.winery[x])]
        data.loc[x,'price'] = filtro.price.median()

### Quedan 4 valores sueltos con valor 0.0 ya que no tienen un precio y no hay otros con esa vineria y variedad <br> Por lo que para llenar estos ultimos hago lo mismo que arriba solo que poniendo menos condiciones solo basandome en la provincia y la variedad <br> Por ejemplo si alguno de estos 4 es Malbec, Mendoza, cambo el 0.0 por la media de todos los vinos Malbec de Mendoza

In [34]:
for x in range(0, len(data)):
    if data.price[x] == 0.0:
        otro_filtro = data[(data.province == data.province[x]) & (data.variety == data.variety[x])]
        data.loc[x,'price'] = otro_filtro.price.median()

### Luego de la funcion se cambio el 0.0 a 15.0 y los demas precios quedaron como estaban por lo que funciona correctamente

In [35]:
datosDos = data[(data.variety == 'chardonnay') & 
    (data.province == 'mendoza') & 
    (data.region_1 == 'valle de uco') &
    (data.winery == 'andeluna')]
datosDos

,index,points,price,region_1,title,variety,winery,categoria,province
513,17155,84,15.0,valle de uco,andeluna 2013 1300 chardonnay (uco valley),chardonnay,andeluna,promotor,mendoza
2151,77855,85,15.0,valle de uco,andeluna 2011 1300 chardonnay (uco valley),chardonnay,andeluna,promotor,mendoza
2476,90905,80,16.0,valle de uco,andeluna 2015 1300 chardonnay (valle de uco),chardonnay,andeluna,neutro,mendoza
2550,93677,84,15.0,valle de uco,andeluna 2012 1300 chardonnay (uco valley),chardonnay,andeluna,promotor,mendoza


### Ya no quedan nulos

In [36]:
data.isnull().sum()

index        0
points       0
price        0
region_1     0
title        0
variety      0
winery       0
categoria    0
province     0
dtype: int64

### Ya tengo el data set ordenado, con filas nuevas, datos completos y solo se eliminaron <span style="color: green">256</span> filas duplicadas no filas unicas que tengan valores NaN

In [37]:
antes_de_drop - despues_de_drop

256

In [38]:
data.rename(columns={'index':'id',
                    'points':'puntaje',
                    'price':'precio_en_pesos',
                    'region_1':'region',
                    'title':'titulo',
                    'variety':'variedad',
                    'winery':'bodega',
                    'province':'provincia'}, 
                     inplace=True)

In [39]:
data.head(10)

,id,puntaje,precio_en_pesos,region,titulo,variedad,bodega,categoria,provincia
0,16,87,30.0,cafayate,felix lavaque 2010 felix malbec (cafayate),malbec,felix lavaque,promotor,salta
1,17,87,13.0,mendoza,gaucho andino 2011 winemaker selection malbec ...,malbec,gaucho andino,promotor,mendoza
2,183,88,12.0,salta,alamos 2007 torrontes (salta),torrontes,alamos,promotor,salta
3,224,90,22.0,lujan de cuyo,mendel 2014 lunta malbec (lujan de cuyo),malbec,mendel,promotor,mendoza
4,231,85,10.0,mendoza,andean sky 2007 bonarda (mendoza),bonarda,andean sky,promotor,mendoza
5,245,85,12.0,san juan,finca las moras 2007 reserve chardonnay (san j...,chardonnay,finca las moras,promotor,san juan
6,253,85,15.0,lujan de cuyo,cueva de las manos 2007 reserve malbec (lujan ...,malbec,cueva de las manos,promotor,mendoza
7,261,89,37.0,mendoza,algodon 2008 estate blend gran reserva red,red blend,algodon,promotor,mendoza
8,266,89,14.0,mendoza,kaiken 2008 corte malbec-bonarda-petit verdot ...,red blend,kaiken,promotor,mendoza
9,273,89,19.0,valle de uco,o. fournier 2007 b crux red (uco valley),red blend,o. fournier,promotor,mendoza


### Ordeno los datos en base al puntaje y al precio descendente

In [40]:
data.sort_values(['puntaje', 'precio_en_pesos'], ascending=[False, False], inplace=True)

In [41]:
data.variedad.unique()

array(['malbec', 'bordeaux-style red blend', 'cabernet sauvignon-malbec',
       'red blend', 'malbec-cabernet sauvignon', 'merlot',
       'cabernet franc-cabernet sauvignon', 'syrah-viognier',
       'cabernet sauvignon', 'malbec-cabernet', 'pinot noir',
       'cabernet franc', 'cabernet blend', 'chardonnay',
       'malbec-cabernet franc', 'petit verdot', 'malbec blend', 'bonarda',
       'cabernet franc-malbec', 'malbec-petit verdot', 'syrah',
       'tempranillo-malbec', 'white blend', 'malbec-merlot',
       'malbec-bonarda', 'shiraz-malbec', 'torrontes', 'nebbiolo',
       'malbec-syrah', 'champagne blend', 'sauvignon blanc',
       'syrah-malbec', 'rosado', 'cabernet sauvignon-merlot', 'trousseau',
       'sparkling blend', 'chardonnay-viognier', 'tannat',
       'cabernet-syrah', 'semillon', 'tempranillo',
       'cabernet sauvignon-shiraz', 'malbec-tannat',
       'merlot-cabernet franc', 'rose', 'cabernet sauvignon-syrah',
       'pinot gris', 'viognier', 'shiraz', 'syrah-b

### Reemplazo las variedades que estan escritas de forma desordenada pero que son iguales a otras.

In [42]:
data.replace({'variedad' : {'malbec-cabernet sauvignon' : 'cabernet sauvignon malbec', 
                            'cabernet sauvignon-malbec' : 'cabernet sauvignon malbec',
                            'malbec-cabernet franc' : 'cabernet franc-malbec', 
                            'malbec-cabernet'  : 'cabernet-malbec', 
                            'cabernet sauvignon-cabernet franc'  : 'cabernet franc-cabernet sauvignon'}}, inplace = True)

### Programacion turbia para obtener un DF con el vino con mayor puntaje por cada marca, 1 por marca

In [43]:
variedades_vinos = data.variedad.unique().tolist()

In [44]:
arreglo = []

In [45]:
for x in range(0, len(variedades_vinos)):
    arreglo.append(data[data.variedad == variedades_vinos[x]].max())

In [46]:
mejores_vinos = pd.DataFrame(arreglo)

In [47]:
mejores_vinos.rename(columns={0 :'id',
                    1 :'puntaje',
                    2 :'precio_en_pesos',
                    3 :'region',
                    4 :'titulo',
                    5 :'variedad',
                    6 :'bodega',
                    7 :'categoria',
                    8 :'provincia'}, 
                     inplace=True)

### Esto era por si el precio estaba en dolares pero esta en pesos asi que lo saque
#### precio_dolar = 43.68 <br> mejores_vinos['precio_en_pesos'] = mejores_vinos['precio_en_dolares'].apply(lambda x: x * precio_dolar)<br>

In [49]:
mejores_vinos

,id,puntaje,precio_en_pesos,region,titulo,variedad,bodega,categoria,provincia
0,129948,97,225.0,vista flores,zuccardi 2015 serie a malbec (valle de uco),malbec,zuccardi,promotor,san juan
1,128326,96,120.0,vista flores,zuccardi 2013 zeta red (uco valley),bordeaux-style red blend,zuccardi,promotor,salta
2,128142,95,169.0,valle de uco,zuccardi 2011 zeta malbec-cabernet sauvignon (...,cabernet sauvignon malbec,zuccardi,promotor,san juan
3,129932,95,135.0,valle de uco,zuccardi 2012 tito red (uco valley),red blend,zuccardi,promotor,san juan
4,128325,94,120.0,valle de uco,zolo 2011 merlot (mendoza),merlot,zolo,promotor,rio negro
5,123132,94,90.0,valle de uco,ricominciare 2009 cabernet franc-cabernet sauv...,cabernet franc-cabernet sauvignon,ricominciare,promotor,san juan
6,76584,94,75.0,mendoza,trapiche 2011 iscay syrah-viognier (mendoza),syrah-viognier,trapiche,promotor,mendoza
7,129617,94,230.0,vista flores,zuccardi 2013 serie a cabernet sauvignon (vall...,cabernet sauvignon,zuccardi,promotor,san juan
8,82920,93,165.0,salta,trivento 2014 reserve cabernet-malbec (mendoza),cabernet-malbec,trivento,promotor,salta
9,128412,93,150.0,vista flores,zorzal 2015 eggo filoso pinot noir (tupungato),pinot noir,zorzal,promotor,salta


### No soy conocedor de vinos asi que tuve que googlear cual era la diferencia entre cabernet sauvignon malbec y cabernet-malbec o cabernet sauvignon asi encontre esto por lo que decidi no modificar esas variedades<br> El Cabernet Sauvignon es el vino tinto conocido en todo el mundo. Variedad que predomina en los vinos Burdeaux de Francia. Sus principales características son: tánico y seco, y en muchas ocasiones con la finalidad de ablandarlo se mezcla con <span style="color: red">Merlot</span>, <span style="color: red">Malbec</span> u otras variedades.

### Agrego el año de creacion obteniendolo del titulo

In [52]:
mejores_vinos["creacion"] = mejores_vinos.titulo.str.extract(r"\b(19\d\d|20[01]\d)\b", expand=False)
mejores_vinos.creacion = mejores_vinos.creacion.fillna(value=0)
mejores_vinos.creacion = mejores_vinos.creacion.astype(int)

### Top 10 de mejores vinos de Argentina en base a su puntaje tomando el que amyor puntaje tenga por variedad

In [54]:
mejores_vinos.head(10)

,id,puntaje,precio_en_pesos,region,titulo,variedad,bodega,categoria,provincia,creacion
0,129948,97,225.0,vista flores,zuccardi 2015 serie a malbec (valle de uco),malbec,zuccardi,promotor,san juan,2015
1,128326,96,120.0,vista flores,zuccardi 2013 zeta red (uco valley),bordeaux-style red blend,zuccardi,promotor,salta,2013
2,128142,95,169.0,valle de uco,zuccardi 2011 zeta malbec-cabernet sauvignon (...,cabernet sauvignon malbec,zuccardi,promotor,san juan,2011
3,129932,95,135.0,valle de uco,zuccardi 2012 tito red (uco valley),red blend,zuccardi,promotor,san juan,2012
4,128325,94,120.0,valle de uco,zolo 2011 merlot (mendoza),merlot,zolo,promotor,rio negro,2011
5,123132,94,90.0,valle de uco,ricominciare 2009 cabernet franc-cabernet sauv...,cabernet franc-cabernet sauvignon,ricominciare,promotor,san juan,2009
6,76584,94,75.0,mendoza,trapiche 2011 iscay syrah-viognier (mendoza),syrah-viognier,trapiche,promotor,mendoza,2011
7,129617,94,230.0,vista flores,zuccardi 2013 serie a cabernet sauvignon (vall...,cabernet sauvignon,zuccardi,promotor,san juan,2013
8,82920,93,165.0,salta,trivento 2014 reserve cabernet-malbec (mendoza),cabernet-malbec,trivento,promotor,salta,2014
9,128412,93,150.0,vista flores,zorzal 2015 eggo filoso pinot noir (tupungato),pinot noir,zorzal,promotor,salta,2015


### Todos los top puntaje de cada variedad ( 67 vinos )

In [57]:
mejores_vinos

,id,puntaje,precio_en_pesos,region,titulo,variedad,bodega,categoria,provincia,creacion
0,129948,97,225.0,vista flores,zuccardi 2015 serie a malbec (valle de uco),malbec,zuccardi,promotor,san juan,2015
1,128326,96,120.0,vista flores,zuccardi 2013 zeta red (uco valley),bordeaux-style red blend,zuccardi,promotor,salta,2013
2,128142,95,169.0,valle de uco,zuccardi 2011 zeta malbec-cabernet sauvignon (...,cabernet sauvignon malbec,zuccardi,promotor,san juan,2011
3,129932,95,135.0,valle de uco,zuccardi 2012 tito red (uco valley),red blend,zuccardi,promotor,san juan,2012
4,128325,94,120.0,valle de uco,zolo 2011 merlot (mendoza),merlot,zolo,promotor,rio negro,2011
5,123132,94,90.0,valle de uco,ricominciare 2009 cabernet franc-cabernet sauv...,cabernet franc-cabernet sauvignon,ricominciare,promotor,san juan,2009
6,76584,94,75.0,mendoza,trapiche 2011 iscay syrah-viognier (mendoza),syrah-viognier,trapiche,promotor,mendoza,2011
7,129617,94,230.0,vista flores,zuccardi 2013 serie a cabernet sauvignon (vall...,cabernet sauvignon,zuccardi,promotor,san juan,2013
8,82920,93,165.0,salta,trivento 2014 reserve cabernet-malbec (mendoza),cabernet-malbec,trivento,promotor,salta,2014
9,128412,93,150.0,vista flores,zorzal 2015 eggo filoso pinot noir (tupungato),pinot noir,zorzal,promotor,salta,2015
